# Cutting Stock Problems (CSP)

1. Gurobipy: [Column](https://www.gurobi.com/documentation/11.0/refman/py_column.html)
2. Gurobipy: [multidict](https://www.gurobi.com/documentation/11.0/refman/py_multidict.html)
3. Gurobipy: [getAttr](https://www.gurobi.com/documentation/11.0/refman/py_model_getattr.html)

In [2]:
#  ============= NOT USING COLUMN GENERATION VERSION ===============
import random
import gurobipy as gp
from gurobipy import GRB
random.seed(42)
# Types  = [3, 7, 9, 16]          # 需求长度
# Demand = [25, 30, 14,8]          # 需求的量
Types  = [] # 需求长度
Demand = [] # 需求的量
num = 100
for i in range(num):
    Types.append(random.randint(2,16))
    Demand.append(random.randint(9,20))
L = 20

K = 0  #设置木材长度数量上限
for i in range(len(Types)):
    K  += Demand[i]//(L//Types[i]) + 2
print(K)
x = {}
y = {}
m = gp.Model("CSP_COMPACT")
for i in range(len(Types)):
    for k in range(K):
        name_x = "x_" + str(i) + "_" + str(k)
        x[i,k] = m.addVar(vtype= GRB.INTEGER,name=name_x)
for k in range(K):
    name_y = "y_" + str(k)
    y[k] = m.addVar(vtype=GRB.BINARY, name=name_y)

m.setObjective(gp.quicksum(y[k] for k in range(K)),GRB.MINIMIZE)
#约束1
for i in range(len(Types)):
    m.addConstr(gp.quicksum(x[i,k] for k in range(K)) >= Demand[i],\
    name = f"c1_i{i}")
#约束2
for k in range(K):
    m.addConstr(gp.quicksum(Types[i]*x[i,k] \
    for i in range(len(Types)))<=L*y[k],name=f"c2_k{k}")
m.setParam(GRB.Param.TimeLimit,600)
m.optimize()
print(m.objval)

1046
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1146 rows, 105646 columns and 210246 nonzeros
Model fingerprint: 0x955c9d34
Variable types: 0 continuous, 105646 integer (1046 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 2e+01]
Presolve time: 0.27s
Presolved: 1146 rows, 105646 columns, 210246 nonzeros
Variable types: 0 continuous, 105646 integer (42886 binary)
Found heuristic solution: objective 669.0000000

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 6.480000e+02, 104602 iterations, 0.19 seconds (0.21 work units)

    Nodes    |    Current Node    |     

In [2]:
def gen_instances(s, o, standard = True):
    """_summary_
    create instances
    Args:
        s (_type_): _description_ # of supply stocks
        o (_type_): _description_ # of order stocks
        standard (bool, optional): _description_. Defaults to True. 是否是标准测试数据(s = 3, o = 8)

    Returns:
        _type_: _description_
    """

    if standard:
        supplies = {
            0: [27, 500],
            1: [42, 900],
            2: [48, 1000],
        }
        orders = {0:[4, 100],
                1:[5,90],
                2:[6,42],
                3:[7,140],
                4:[9,90],
                5:[17,55], 
                6:[21,53],
                7:[29,44]} 
        return supplies, orders
    else:
        s = min(s, 10)
        random.seed(42)
        supplies = dict()
        INIT_supplies = {
            0: [135, 500],
            1: [210, 900],
            2: [240, 1000],
            3: [335, 1400],
            4: [600, 3000],
            5: [750, 3400],
            6: [800, 3800],
            7: [1000, 4500],
            8: [1200, 5300],
            9: [1500, 6100],
        }
        for i in range(s):
            supplies[i] = INIT_supplies[i]
        orders = dict() # 需求的量
        Demand = []
        # Types = list(random.sample(range(50, 200), o))
        Types = list(random.sample(range(20, 580), o))
        Demand = list(random.sample(range(200, 800), o))
        for i in range(o):
            orders[i] = [Types[i], Demand[i]]
        return supplies, orders


In [4]:
# ===================== USING COLUMN GENERATION ALGORITHM ====================
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB
import math


class MasterProblem:
    def __init__(self):
        self.model = gp.Model("master")
        self.vars = None
        self.constrs = None
        self.objCoef = None
        self.patternIndex = None
        
    def setup(self, patterns, demand):
        num_patterns = len(patterns)
        self.vars = self.model.addVars(num_patterns, obj=self.objCoef, name="Pattern")
        self.constrs = self.model.addConstrs((gp.quicksum(patterns[pattern][piece]*self.vars[pattern]
                                                          for pattern in range(num_patterns))
                                              >= demand[piece] for piece in demand.keys()),
                                             name="Demand")
        self.model.modelSense = GRB.MINIMIZE
        self.model.params.outputFlag = 0
        self.model.update()
        
    def update(self, pattern, index, objcoef, patternIndex):
        new_col = gp.Column(coeffs=pattern, constrs=self.constrs.values())
        self.vars[index] = self.model.addVar(obj=objcoef, column=new_col,
                                             name=f"Pattern[{index}]")
        self.patternIndex.append(patternIndex)
        self.model.update()


class SubProblem:
    """_summary_
    子问题实际上是一个背包问题
    """
    def __init__(self):
        self.model = gp.Model("subproblem")
        self.vars = {}
        self.constr = None
        
    def setup(self, stock_length, lengths, duals):
        self.vars = self.model.addVars(len(lengths), obj = duals, vtype = GRB.INTEGER,
                                       name="Frequency")
        
        self.constr = self.model.addConstr(self.vars.prod(lengths) <= stock_length,
                                           name="Knapsack")
        self.model.modelSense = GRB.MAXIMIZE
        self.model.params.outputFlag = 0
        # Stop the subproblem routine as soon as the objective's best bound becomes
        #less than or equal to one, as this implies a non-negative reduced cost for
        #the entering column.
        self.model.params.bestBdStop = 1
        self.model.update()
        
    def update(self, duals):
        self.model.setAttr("obj", self.vars, duals)
        self.model.update()


class CuttingStock:
    def __init__(self, supplies, pieces):
        self.stock_pieces, self.stock_lengths, self.stock_cost = gp.multidict(supplies) # 解析原料钢管的数据
        self.pieces, self.lengths, self.demand = gp.multidict(pieces) # multidict 每个键都应映射到值列表。
        # pieces: 需求k的编号
        # lengths: 需求k的钢管的长度
        # demand: 需求k的钢管的数量
        self.patterns = None # 所有的模式
        self.duals = [0]*len(self.pieces) # 对偶成本 数量 == 约束的个数
        piece_reqs = [length*req for length, req in pieces.values()] # 每个需求的总长度（长度 x 数量）
        # self.min_rolls = np.ceil(np.sum(piece_reqs)/stock_length) # 
        self.solution = {}
        self.master = MasterProblem()
        self.subproblem = [SubProblem() for _ in range(len(self.stock_pieces))]
        
    def _initialize_patterns(self):
        # 贪心地进行初始化：用每一根原料切每一种产品，依此作为切割方式
        patterns = []
        objcoefs = []
        patternIndex = []
        for i, stock_length_sub in self.stock_lengths.items(): 
        # 要多增加一个循环
            for idx, length in self.lengths.items():
                pattern = [0]*len(self.pieces)
                pattern[idx] = stock_length_sub // length
                patterns.append(pattern)
                objcoefs.append(self.stock_cost[i])
                patternIndex.append(i)
        self.patterns = patterns
        self.master.objCoef = objcoefs
        self.master.patternIndex = patternIndex
        
    def _generate_patterns(self):
        self._initialize_patterns()
        self.master.setup(self.patterns, self.demand)
        for i in range(len(self.stock_pieces)):
            self.subproblem[i].setup(self.stock_lengths[i], self.lengths, self.duals)
        while True:
            self.master.model.optimize()
            self.duals = self.master.model.getAttr("pi", self.master.constrs) # 当前主问题的dual Price
            temp_pattern = [0] * len(self.pieces)
            temp_reducedCost = 99999
            temp_objcoef = 0
            temp_subpatternIndex = -1
            stop_iter = True
            for i  in range(len(self.stock_pieces)):
                # 从每个子问题里找到Reduced cost 最小的那个 每次只把提升最大的这一列加进去
                # 也可以把每个Reduced cost < 0的都加进去
                self.subproblem[i].update(self.duals)
                self.subproblem[i].model.optimize()
                reduced_cost = self.stock_cost[i]  - self.subproblem[i].model.objVal
                if reduced_cost < 0:
                    stop_iter = False
                    if reduced_cost < temp_reducedCost:
                        temp_reducedCost = reduced_cost
                        pattern = [0]*len(self.pieces)
                        for piece, var in self.subproblem[i].vars.items():
                            if var.x > 0.5:
                                pattern[piece] = math.ceil(var.x)
                        temp_pattern = pattern
                        temp_objcoef = self.stock_cost[i]
                        temp_subpatternIndex = i
            if stop_iter:
                # 如果每个子问题的检验数都是不小于 0 的， 说明当前已经找到最优解了
                break
            else:
                # 否则更新主问题的结构
                self.master.update(temp_pattern, len(self.patterns), temp_objcoef, temp_subpatternIndex)
                self.patterns.append(temp_pattern)
    
    def pattern_usage(self, sol):
        return sum([self.lengths[i] * self.patterns[sol][i]  for i in range(len(self.pieces))])

    def solve(self):
        self._generate_patterns()
        self.master.model.setAttr("vType", self.master.vars, GRB.INTEGER)
        self.master.model.optimize()
        ReObjVal = 0
        if self.master.model.status == GRB.OPTIMAL:
            idx = 0
            for pattern, var in self.master.vars.items():
                if var.x > 1e-6:
                    self.solution[pattern] = math.ceil(var.x)
                    ReObjVal += self.solution[pattern] * \
                        self.stock_cost[self.master.patternIndex[pattern]]
                elif var.x > 0 and var.x < 0.5:
                    print("Less than 0.5! variables")
            idx += 1
        print(f"修正后的目标函数（总成本）: {ReObjVal}")
        

if __name__ == "__main__": 
    s, o, standard = 5, 200, False
    supplies, orders = gen_instances(s, o, standard)
    
    NUM = len(orders)
    mycsp = CuttingStock(supplies, orders)
    mycsp.solve()
    print(f"测试案例：原料：{s}, 订单：{o}")
    print(f"总成本：{mycsp.master.model.ObjVal}")
    print(f"总列数（决策变量数）: {len(mycsp.master.model.getVars())}")
    print(f"总约束（需求的木材种类）：{len(mycsp.master.model.getConstrs())}")
    actualSupplies = {i : 0 for i in range(NUM)}
    for sol in mycsp.solution:
        print(f"{mycsp.patterns[sol] } ，原料长度 / 切割长度： {mycsp.stock_lengths[mycsp.master.patternIndex[sol]]} / {mycsp.pattern_usage(sol)}, 产量 : {mycsp.solution[sol]}")
        for n in range(NUM):
            if mycsp.patterns[sol][n] > 0.5:
                actualSupplies[n] += (mycsp.solution[sol] * mycsp.patterns[sol][n])
    for n in range(NUM):
        print(f"切割物件{n}的需求：{orders[n][1]}, 物件长度：{orders[n][0]}， 实际产量：{actualSupplies[n]}, 差额：{actualSupplies[n] - orders[n][1]}")


# 3, 50 11566900 INTEGER 0.03s
# 3, 50 11565700 INTEGER 0.09s
# Gap: 0.10 %

# 3,100, 25638500 CONTINUOUS 0.2s
# 3,100, 25638000 INTEGER 0.2s
# Gap: 0.001 %


# 4,100, 31886800 CONTINUOUS 0.3s
# 4,100, 31868800 INTEGER 0.3 s
# Gap: 0.6 %


# 5,200: 154240900 CONTINUOUS  1.1
# 5,200: 154228300 INTEGER 1.2 s
# Gap: 0.008 % 

# 6,100: 62403100 CONTINUOUS 5.2s
# 6,100: 62375400 INTEGER 5.3s
# Gap: 0.04 %

# 6,150: 94389600 CONTINUOUS 8.7s
# 6,150: 94334200 INTEGER 8.7s
# Gap: 0.06 %

# 6,200: 144396800 CONTINUOUS 18.6s
# 6,200: 144293200 INTEGER 18.6s
# Gap: 0.07%

# 5,300: 230901200 CONTINUOUS 2.3 s
# 5,300: 230789800 INTEGER 2.4 s
# Gap: 0.04 % 

# 5,400: 314893400 CONTINUOUS  4.7s
# 5,400: 314778800 INTEGER  4.7 s
# Gap: 0.03 % 

# 5,500 374405300 CONTINUOUS 11.3s 
# 5,500 374331600 INTEGER 11.5s
# Gap: 0.02 %

修正后的目标函数（总成本）: 144396800
测试案例：原料：6, 订单：200
总成本：144279673.17001185
总列数（决策变量数）: 1787
总约束（需求的木材种类）：200
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ，原料长度 / 切割长度： 135 / 134, 产量 : 458
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 